In [317]:
import pandas as pd
import numpy as np
import os, sys

# Now you can import the library as usual
import investos as inv
from scipy.stats import ttest_ind
from sklearn.linear_model import LinearRegression
from datetime import datetime
from scipy.stats.mstats import winsorize
import matplotlib.pyplot as plt
import seaborn as sns
from tabulate import tabulate
from sklearn.preprocessing import StandardScaler

import statsmodels.api as sm

In [631]:
# Step 1: loading the dataframe and join them together
# loading
df_loadings = pd.read_parquet('john_rbics_industry.parquet')
df_actual_returns = pd.read_parquet('john_return_prev_1d_open.parquet').rename(columns={"value": "return_1d"})
df_sizes = pd.read_parquet('john_market_cap_open_dil.parquet').rename(columns={"value": "size"})
# log and then winsorize
df_sizes['size'] = np.log(df_sizes['size'])
df_values = pd.read_parquet('john_market_cap_open_dil_to_net_income_ltm.parquet').rename(columns={"value": "value"})
df_loadings = pd.get_dummies(df_loadings, columns=["industry"])

# merge these dataframe into one
df_values = df_values.merge(df_sizes, how='left', on=['id','datetime'])
df_actual_returns = df_actual_returns.merge(df_values, how='left', on=['id','datetime'])
df_loadings = pd.merge(df_actual_returns, df_loadings, how='left', on=None, left_on='id', right_on='id', left_index=False)

for col in df_loadings.columns:
    if col.startswith("industry_"):
        df_loadings[col] = df_loadings[col].astype(int)

In [632]:
df_loadings[(df_loadings.id == 'XR7GZL-R') & (df_loadings.datetime == '2024-01-31')]

,id,datetime,return_1d,value,size,industry_Business Services,industry_Consumer Cyclicals,industry_Consumer Non-Cyclicals,industry_Consumer Services,industry_Energy,industry_Finance,industry_Healthcare,industry_Industrials,industry_Non-Corporate,industry_Non-Energy Materials,industry_Other,industry_Technology,industry_Telecommunications,industry_Utilities
24411632,XR7GZL-R,2024-01-31,0.013623,20.462871,7.755486,0,0,0,0,0,1,0,0,0,0,0,0,0,0


In [633]:
# Step 2: Calculate momentum (e.g., 12-month cumulative return)

# sliding window size for volatiltiy and momentum 
#look_back_period_volatility = 252  # Approx. 12 months for daily data
look_back_period_momentum = 30 
#df_loadings['momentum'] = df_loadings.groupby('id')['return_1d'].apply(lambda x: x.rolling(window=look_back_period_momentum).apply(lambda y: np.prod(1 + y) - 1))
#df_loadings['volatility'] = df_loadings.groupby('id')['return_1d'].apply(lambda x: x.rolling(window=look_back_period_volatility).std()).reset_index(level = 'id').rename(columns={"return_1d": "volatility"}).volatility
df_loadings['momentum'] = df_loadings.groupby('id')['return_1d'].rolling(window=look_back_period_momentum).apply(lambda x: np.prod(1 + x) - 1, raw = True).reset_index(level = 'id').rename(columns={"return_1d": "momentum"}).momentum
# drop those nan values less than then window size
df_loadings['momentum'] = df_loadings['momentum'].shift(periods = 1)
df_loadings['country'] = 1

df_loadings.dropna(inplace = True)
df_loadings.replace([np.inf, -np.inf], 0, inplace=True)

In [634]:
df_loadings[(df_loadings.id == 'XR7GZL-R') & (df_loadings.datetime == '2024-01-31')]

,id,datetime,return_1d,value,size,industry_Business Services,industry_Consumer Cyclicals,industry_Consumer Non-Cyclicals,industry_Consumer Services,industry_Energy,...,industry_Healthcare,industry_Industrials,industry_Non-Corporate,industry_Non-Energy Materials,industry_Other,industry_Technology,industry_Telecommunications,industry_Utilities,momentum,country
24411632,XR7GZL-R,2024-01-31,0.013623,20.462871,7.755486,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0.09365,1


In [635]:
# Step 3. Winsorize and standardize

# Identify numeric columns excluding 'id' and 'datetime'
startDate = '2020-01-01'
df_loadings = df_loadings[df_loadings.datetime >= startDate]
cols = [col for col in df_loadings if col not in ['datetime','id','country','return_1d','momentum','period'] and not col.startswith('industry_')]

# Function to winsorize and standardize within each group
def process_group(group):
    for col in cols:
        # Winsorizing numeric columns at 5th and 95th percentiles within the group
        group[col] = winsorize(group[col], limits=[0.05, 0.05])
    
    # Ensure no infinite values are left
    group[cols] = np.nan_to_num(group[cols], nan=0.0, posinf=0.0, neginf=0.0)
    
    # Standardizing numeric columns within the group
    scaler = StandardScaler()
    group[cols] = scaler.fit_transform(group[cols])
    
    return group

# Apply the processing function to each group defined by 'datetime'
df_loadings = df_loadings.groupby('datetime').apply(process_group, include_groups=False)
df_loadings = df_loadings.reset_index().drop(columns=['level_1'])

In [636]:
df_loadings[(df_loadings.id == 'XR7GZL-R') & (df_loadings.datetime == '2024-01-31')]

,datetime,id,return_1d,value,size,industry_Business Services,industry_Consumer Cyclicals,industry_Consumer Non-Cyclicals,industry_Consumer Services,industry_Energy,...,industry_Healthcare,industry_Industrials,industry_Non-Corporate,industry_Non-Energy Materials,industry_Other,industry_Technology,industry_Telecommunications,industry_Utilities,momentum,country
3383986,2024-01-31,XR7GZL-R,0.013623,0.281468,-0.007366,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0.09365,1


In [637]:
#df_loadings

In [638]:
# identify factor effectiveness
# Step 1: calculate t statistics 
def get_t_statistics(model,X,y):
    predictions = model.predict(X)     
    residuals = y - predictions
    # Calculate the residual standard error
    rss = (residuals ** 2).sum()

    n = len(y)      # number of sample
    p = X.shape[1]  # number of factors
    rmse = (rss/n)**0.5
    rse = (rss / (n - p - 1)) ** 0.5
    # Calculate t-values for coefficients
    t_values = model.coef_ / (rse/n**0.5)

    return t_values, residuals

In [639]:
# cols = ['industry_Business Services', 'industry_Consumer Cyclicals', 'industry_Consumer Non-Cyclicals', 'industry_Consumer Services', 'industry_Energy', 'industry_Finance', 'industry_Healthcare', 'industry_Industrials', 'industry_Non-Corporate', 'industry_Non-Energy Materials', 'industry_Other', 'industry_Technology', 'industry_Telecommunications', 'industry_Utilities']

# df_current[cols]

In [641]:

"""
1.
it comes out with an warning like this:
# /var/folders/sg/1nh38dx53_zc3kxcq1cq1f000000gn/T/ipykernel_29543/1917117034.py:11: RuntimeWarning: invalid value encountered in divide
#  t_values = model.coef_ / rse

this is caused by t_values = model.coef_ / rse while certain model.coef_ value is too low (about e-17 level). Overall the code is fine.
We can just mute this warning later

2. some results shows [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.] 0.0 which model.coef_ is a zero list, and rse is zero 
as well
"""
cols = [col for col in df_loadings.columns if col.startswith("industry_")] + ['momentum', 'size', 'value']

factor_returns = {}
# change to freq 2Y to any period you want
df_loadings["period"] = df_loadings["datetime"].dt.to_period(freq = 'Y')

for d in df_loadings["datetime"].unique():
    # Isolating the data for the current date
    df_current = df_loadings[df_loadings["datetime"] == d]

    # outlier modification and factor standarization
    # identify factor effectiveness
    y = df_current["return_1d"]
    X = df_current[cols]

    #Performing linear regression
    model = LinearRegression(fit_intercept=False).fit(X, y)

    t_values, residuals = get_t_statistics(model,X,y)
    #model = sm.OLS(y, X).fit()
    #print(model.summary())
    # Print t-values for each factor
    # print("T-Values:")
    # for i, factor in enumerate(X.columns):
    #     print(f"{factor}: {t_values[i]}")
    
    # Storing the coefficients and intercept
    factor_returns[d] = {
        "coefficients": model.coef_,
        "intercept": model.intercept_,
        "feature_names": model.feature_names_in_,
        "r2": model.score(X, y),
         # t_values added to the factor_returns dataframe
        "t-values": abs(t_values),
        "residuals": residuals,
        "period":df_current.period.unique()
    }

/var/folders/sg/1nh38dx53_zc3kxcq1cq1f000000gn/T/ipykernel_54770/2123765357.py:14: RuntimeWarning: invalid value encountered in divide
  t_values = model.coef_ / (rse/n**0.5)
/var/folders/sg/1nh38dx53_zc3kxcq1cq1f000000gn/T/ipykernel_54770/2123765357.py:14: RuntimeWarning: invalid value encountered in divide
  t_values = model.coef_ / (rse/n**0.5)
/var/folders/sg/1nh38dx53_zc3kxcq1cq1f000000gn/T/ipykernel_54770/2123765357.py:14: RuntimeWarning: invalid value encountered in divide
  t_values = model.coef_ / (rse/n**0.5)
/var/folders/sg/1nh38dx53_zc3kxcq1cq1f000000gn/T/ipykernel_54770/2123765357.py:14: RuntimeWarning: invalid value encountered in divide
  t_values = model.coef_ / (rse/n**0.5)
/var/folders/sg/1nh38dx53_zc3kxcq1cq1f000000gn/T/ipykernel_54770/2123765357.py:14: RuntimeWarning: invalid value encountered in divide
  t_values = model.coef_ / (rse/n**0.5)
/var/folders/sg/1nh38dx53_zc3kxcq1cq1f000000gn/T/ipykernel_54770/2123765357.py:14: RuntimeWarning: invalid value encountered 

In [642]:
factor_returns

{Timestamp('2020-01-01 00:00:00'): {'coefficients': array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]),
  'intercept': 0.0,
  'feature_names': array(['industry_Business Services', 'industry_Consumer Cyclicals',
         'industry_Consumer Non-Cyclicals', 'industry_Consumer Services',
         'industry_Energy', 'industry_Finance', 'industry_Healthcare',
         'industry_Industrials', 'industry_Non-Corporate',
         'industry_Non-Energy Materials', 'industry_Other',
         'industry_Technology', 'industry_Telecommunications',
         'industry_Utilities', 'momentum', 'size', 'value'], dtype=object),
  'r2': 1.0,
  't-values': array([nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
         nan, nan, nan, nan]),
  'residuals': 0       0.0
  1       0.0
  2       0.0
  3       0.0
  4       0.0
         ... 
  3039    0.0
  3040    0.0
  3041    0.0
  3042    0.0
  3043    0.0
  Name: return_1d, Length: 3044, dtype: float64,
  'period': <Pe

In [580]:
list_to_insert = [
    [k, factor_returns[k]["r2"], *factor_returns[k]["t-values"],*factor_returns[k]["coefficients"], factor_returns[k]["period"][0]]
    for k in factor_returns    # iterative through the factor returns dict
]
#print(list_to_insert)
#cols = df_loadings.drop(columns=["return_1d", "datetime", "id"]).columns
#cols = ['country','size','value','momentum']
#cols = factor_returns[0]['feature_names']
# update
cols_with_return = ['returns_' + col for col in cols]
cols_with_t_values = ['t_values_' + col for col in cols]

# df_factor_returns = df.append(pd.Series(list_to_insert, index=['date', 'r2', *cols]), ignore_index=True)  # using append
df_factor_returns = pd.DataFrame(list_to_insert, columns=["datetime", "r2", *cols_with_t_values, *cols_with_return, "period"])

# generate df_returns_residual df


In [660]:
residual_list = [factor_returns[k]["residuals"] for k in factor_returns]
pd.concat(residual_list)
df_residuals = df_loadings[["datetime","id"]].merge(pd.concat(residual_list), left_index = True, right_index = True).rename(columns = {"return_1d":"residuals"})

In [661]:
df_residuals

,datetime,id,residuals
0,2020-01-01,B00FG1-R,0.000000
1,2020-01-01,B01HWF-R,0.000000
2,2020-01-01,B04XY5-R,0.000000
3,2020-01-01,B06HD3-R,0.000000
4,2020-01-01,B0TXKG-R,0.000000
...,...,...,...
3383982,2024-01-31,XQF8XY-R,0.003404
3383983,2024-01-31,XQFML8-R,0.015875
3383984,2024-01-31,XQN3ZC-R,0.024537
3383985,2024-01-31,XR00QK-R,0.008522


In [663]:
for d in df_residuals["datetime"].unique():
    print(df_residuals[df_residuals["datetime"] == d]['residuals'].var())
    

0.0
0.0006653017687210244
0.0006981677642964903
0.0006611820550913612
0.0007757355050212405
0.0005418296086203554
0.0007031649753673212
0.0010254592578787217
0.0006983707939816298
0.0010296164067895846
0.0007937146928843099
0.001074318336425916
0.0006634279073346698
0.0
0.0007927294891001571
0.0008663402973727766
0.001115548015581239
0.0004845033233677611
0.0024912372049648674
0.00085348555320735
0.0009322672522507455
0.0008436013479148111
0.0008039930156901618
0.0006661581818018503
0.0007856469121024109
0.0007845300095898771
0.0012934525874673658
0.0013848103819029778
0.0006245619168411438
0.0009560450133161686
0.0007846405174928273
0.00076382688764746
0.0009634858867780253
0.0
0.0006781905896986496
0.0009996492971195746
0.001124896065839523
0.0011750710768371315
0.001168160585199069
0.0023285578739012836
0.001174804240787132
0.001816916812251063
0.0028180507506176435
0.0022693079448045045
0.0015001558140794232
0.0012473435511671945
0.0008909752109166529
0.002024644859161111
0.0039218

In [647]:
pd.DataFrame(pd.concat(residual_list),columns = ['residuals'])

,residuals


In [649]:
type(pd.concat(residual_list))

pandas.core.series.Series

In [581]:
df_factor_returns.columns

Index(['datetime', 'r2', 't_values_industry_Business Services',
       't_values_industry_Consumer Cyclicals',
       't_values_industry_Consumer Non-Cyclicals',
       't_values_industry_Consumer Services', 't_values_industry_Energy',
       't_values_industry_Finance', 't_values_industry_Healthcare',
       't_values_industry_Industrials', 't_values_industry_Non-Corporate',
       't_values_industry_Non-Energy Materials', 't_values_industry_Other',
       't_values_industry_Technology', 't_values_industry_Telecommunications',
       't_values_industry_Utilities', 't_values_momentum', 't_values_size',
       't_values_value', 'returns_industry_Business Services',
       'returns_industry_Consumer Cyclicals',
       'returns_industry_Consumer Non-Cyclicals',
       'returns_industry_Consumer Services', 'returns_industry_Energy',
       'returns_industry_Finance', 'returns_industry_Healthcare',
       'returns_industry_Industrials', 'returns_industry_Non-Corporate',
       'returns_indu

In [582]:
industry_cols = [col for col in df_factor_returns.columns if col.startswith('t_values_')] + ['r2']
df_factor_returns[industry_cols].mean(axis = 0)

t_values_industry_Business Services         1.772860e+01
t_values_industry_Consumer Cyclicals        2.420559e+01
t_values_industry_Consumer Non-Cyclicals    1.674066e+01
t_values_industry_Consumer Services         2.305906e+01
t_values_industry_Energy                    2.995068e+01
t_values_industry_Finance                   1.888085e+01
t_values_industry_Healthcare                2.228077e+01
t_values_industry_Industrials               2.155898e+01
t_values_industry_Non-Corporate             8.304093e-15
t_values_industry_Non-Energy Materials      2.112414e+01
t_values_industry_Other                     6.257786e-15
t_values_industry_Technology                2.289067e+01
t_values_industry_Telecommunications        1.747097e+01
t_values_industry_Utilities                 1.548236e+01
t_values_momentum                           2.646948e+01
t_values_size                               4.099401e+00
t_values_value                              2.305364e+00
r2                             

In [583]:
df_factor_returns.groupby('period').mean()

,datetime,r2,t_values_industry_Business Services,t_values_industry_Consumer Cyclicals,t_values_industry_Consumer Non-Cyclicals,t_values_industry_Consumer Services,t_values_industry_Energy,t_values_industry_Finance,t_values_industry_Healthcare,t_values_industry_Industrials,...,returns_industry_Industrials,returns_industry_Non-Corporate,returns_industry_Non-Energy Materials,returns_industry_Other,returns_industry_Technology,returns_industry_Telecommunications,returns_industry_Utilities,returns_momentum,returns_size,returns_value
period,,,,,,,,,,,,,,,,,,,,,
2020,2020-07-01 16:45:48.091602944,0.109220,19.136968,24.005417,16.869357,25.431078,32.528274,21.482666,19.844433,21.604769,...,0.000655,6.095534e-19,0.001441,-6.926272e-20,0.001775,0.001013,-0.000150,-0.003966,0.000052,-0.000249
2021,2021-07-02 19:07:35.172413696,0.109318,16.898568,23.592889,15.091105,21.952353,31.412535,16.674006,21.551373,21.096390,...,0.000894,-1.232316e-18,0.001305,3.572467e-19,0.000864,0.000980,0.000422,-0.001525,0.000128,0.000150
2022,2022-07-02 12:00:00.000000000,0.126439,21.037716,30.214536,21.023334,28.620449,35.010632,20.268490,30.363273,25.673875,...,-0.001571,-2.100715e-17,0.000304,-1.598354e-17,-0.000038,-0.003112,0.005951,-0.095238,-0.002781,-0.002093
2023,2023-07-01 12:00:00.000000000,0.089313,14.247296,19.440667,14.265594,16.798187,21.778209,17.523644,17.565662,18.240132,...,0.001083,3.258006e-19,0.000620,-4.485365e-19,0.001284,0.000086,-0.000380,-0.001843,0.000349,-0.000217
2024,2024-01-15 15:39:07.826086912,0.149477,12.613654,18.873785,13.261683,15.824696,18.167513,13.589075,19.912635,16.885486,...,-0.001415,-4.855340e-19,-0.002719,-8.001883e-19,-0.000912,0.000141,-0.002878,-0.002635,0.001047,0.000176


In [584]:
df_factor_returns.head()

,datetime,r2,t_values_industry_Business Services,t_values_industry_Consumer Cyclicals,t_values_industry_Consumer Non-Cyclicals,t_values_industry_Consumer Services,t_values_industry_Energy,t_values_industry_Finance,t_values_industry_Healthcare,t_values_industry_Industrials,...,returns_industry_Non-Corporate,returns_industry_Non-Energy Materials,returns_industry_Other,returns_industry_Technology,returns_industry_Telecommunications,returns_industry_Utilities,returns_momentum,returns_size,returns_value,period
0,2020-01-01,1.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.000000e+00,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2020
1,2020-01-02,0.042633,22.353142,29.666148,34.181877,20.448300,45.521730,17.333699,36.477993,24.854178,...,-8.673617e-19,0.012262,3.469447e-18,0.016917,0.010388,0.005673,0.001771,-0.001989,-0.001348,2020
2,2020-01-03,0.068730,20.262788,34.308947,46.077407,25.902691,8.163698,30.106112,49.567043,25.704554,...,-1.734723e-18,-0.013419,-3.469447e-18,-0.005780,-0.011253,-0.015129,-0.001534,0.000423,0.000551,2020
3,2020-01-06,0.013387,1.825378,8.536786,1.123627,6.119165,1.134268,4.886859,0.583701,10.976945,...,-1.301043e-18,-0.004573,-8.673617e-19,-0.001600,-0.003968,0.004443,0.001861,-0.000513,0.000512,2020
4,2020-01-07,0.062333,15.416070,18.755486,9.685412,15.016929,20.279447,6.649213,35.428669,10.122473,...,5.204170e-18,-0.002166,-3.469447e-18,0.016135,0.007271,-0.001347,-0.001786,-0.002363,-0.001082,2020


#### 4.4 Calculate CoVar Factor Matrix

In [590]:
from typing import Sequence

import numpy as np
from matplotlib import pyplot as plt
from sklearn.neighbors import KernelDensity


def ewa(arr: Sequence, half_life: int | None = None) -> float | np.ndarray:
    """Exponential Weighted Average (EWA)

    Parameters
    ----------
    arr : Sequence
        Array of numbers or arrays (later one on axis=0 weights more)
    half_life : int | None, optional
        Steps it takes for weight to reduce to half of the original value, by default
        None, meaning that weights are all equal

    Returns
    -------
    float | np.ndarray
        Exponential weighted average of elements in `arr`
    """
    arr = np.array(arr)
    alpha = 1.0 if half_life is None else 0.5 ** (1 / half_life)
    weights = alpha ** np.arange(len(arr) - 1, -1, -1)
    w_shape = tuple([arr.shape[0]] + [1] * (len(arr.shape) - 1))
    weights = weights.reshape(w_shape)
    sum_weight = len(arr) - 1 if half_life is None else np.sum(weights)
    return (weights * arr).sum(axis=0) / sum_weight


def cov_ewa(data: np.ndarray, half_life: int | None = None, lag: int = 0) -> np.ndarray:
    """Calculate the covariance matrix as an exponential weighted average of range

    Parameters
    ----------
    data : np.ndarray
        Data matrix (K features * T periods)
    half_life : int | None, optional
        Argument in ewa(), by default None
    lag : int, optional
        Difference between to terms of fator, cov(t-lag, t), when lag is opposite, the
        result is transposed, by default 0

    Returns
    -------
    np.ndarray
        Covariance matrix
    """
    if not isinstance(data, np.ndarray):
        raise Exception("data matrix should be an ndarray or pd.DataFrame")
    if data.shape[0] > data.shape[1]:
        raise Exception("data matrix should not have less columns than rows")
    if lag >= data.shape[1]:
        raise Exception("lag must be smaller than the number of columns of matrix")
    data = data.astype("float64")
    f_bar = data.mean(axis=1)
    data = data - f_bar.reshape(data.shape[0], -1)
    t_range = range(lag, data.shape[1]) if lag > 0 else range(data.shape[1] + lag)
    #print(t_range)
    elements = np.array([np.outer(data[:, t - lag], data[:, t]) for t in t_range])
    return ewa(elements, half_life)

In [591]:
# import numpy as np
# from bias_statistics import BiasStatsCalculator
# from utils import cov_ewa


class FactorCovAdjuster:
    """Adjustments on factor covariance matrix"""

    def __init__(self, FRM: pd.DataFrame, window: int | None = None) -> None:
        """Initialization

        Parameters
        ----------
        FRM : np.ndarray
            Factor return matrix (T*K)
        """
        self.T, self.K = FRM.shape
        if self.K > self.T:
            raise Exception("number of periods must be larger than number of factors")
        self.FRM = FRM.astype("float64")

        if window and window > self.T:
            raise Exception("number of window must be larger than number of periods")
    
        if window:

            self.window = window
            self.first_level_index = FRM.index[self.window-1:]
        else:
            self.first_level_index = FRM.index
            self.window = 0
            
        self.FCM = None
        self.second_level_index = FRM.columns
        self.first_level_index = FRM.index[self.window-1:]

    def calc_fcm_raw(self, half_life: int | None = None) -> pd.DataFrame:
        """Calculate the factor covariance matrix, FCM (K*K)

        Parameters
        ----------
        half_life : int
            Steps it takes for weight in EWA to reduce to half of the original value

        Returns
        -------
        np.ndarray
            FCM, denoted by `F_Raw`
        """
        cov_matrices = []
        if not self.window:
            cov_matrices.append(cov_ewa(self.FRM.T.to_numpy(), half_life))
        else:
            for i in range(len(self.FRM)-window+1):
                cov_matrices.append(cov_ewa(self.FRM.iloc[i:i+window,:].T.to_numpy(), half_life))
                #print(cov_ewa(test.iloc[i:i+window,:].T.to_numpy()))
        cov_matrices = np.array(cov_matrices)
        #print(cov_matrices)
        #self.FCM = cov_ewa(self.FRM, half_life).astype("float64")
        multi_index = pd.MultiIndex.from_product([self.first_level_index, self.second_level_index], names=['datetime', 'factor'])
        #print(multi_index)
        return pd.DataFrame(cov_matrices.reshape((-1, cov_matrices.shape[-1])), index = multi_index, columns = self.second_level_index)


    def newey_west_adjust(self,
        FRM: np.ndarray, half_life: int, max_lags: int, multiplier: int
    ) -> np.ndarray:
        """Apply Newey-West adjustment on `F_Raw`

        Parameters
        ----------
        half_life : int
            Steps it takes for weight in EWA to reduce to half of the original value
        max_lags : int
            Maximum Newey-West correlation lags
        multiplier : int
            Number of periods a FCM with new frequence contains

        Returns
        -------
        np.ndarray
            Newey-West adjusted FCM, denoted by `F_NW`
        """
        FCM = 0
        for D in range(1, max_lags + 1):
            C_pos_delta = cov_ewa(FRM, half_life, D)
            FCM += (1 - D / (1 + max_lags)) * (C_pos_delta + C_pos_delta.T)
        D, U = np.linalg.eigh(FCM * multiplier)
        D[D <= 0] = 1e-14  # fix numerical error
        self.FCM = U.dot(np.diag(D)).dot(U.T)
        D, U = np.linalg.eigh(FCM)
        return FCM

    def calc_newey_west_frm(self, max_lags:int, multiplier: int, half_life: int | None = None) -> pd.DataFrame:

        cov_matrices = []
        if not self.window:
            cov_matrices.append(self.newey_west_adjust(self.FRM.T.to_numpy(), 
                                                  half_life = half_life, 
                                                  max_lags = max_lags, 
                                                  multiplier = multiplier))
        else:
            for i in range(len(self.FRM)-window+1):
                cov_matrices.append(self.newey_west_adjust(self.FRM.iloc[i:i+window,:].T.to_numpy(), 
                                            half_life = half_life,
                                            max_lags = max_lags, 
                                            multiplier = multiplier))

        cov_matrices = np.array(cov_matrices)
        #print(cov_matrices)
        #self.FCM = cov_ewa(self.FRM, half_life).astype("float64")
        multi_index = pd.MultiIndex.from_product([self.first_level_index, self.second_level_index], names=['datetime', 'factor'])
        #print(multi_index)
        return pd.DataFrame(cov_matrices.reshape((-1, cov_matrices.shape[-1])), index = multi_index, columns = self.second_level_index)



In [592]:
test.shape

(1066, 17)

In [593]:
factorcovadjuster = FactorCovAdjuster(test)

In [594]:
type(factorcovadjuster.FRM.T.to_numpy())

numpy.ndarray

In [595]:
factorcovadjuster.calc_fcm_raw()

returns_industry_Business Services  \
datetime   factor                                                                        
2024-01-31 returns_industry_Business Services                             2.076908e-04   
           returns_industry_Consumer Cyclicals                            2.585162e-04   
           returns_industry_Consumer Non-Cyclicals                       -2.642917e-04   
           returns_industry_Consumer Services                             2.562457e-04   
           returns_industry_Energy                                        2.706373e-04   
           returns_industry_Finance                                       2.182466e-04   
           returns_industry_Healthcare                                    1.844511e-04   
           returns_industry_Industrials                                   2.316459e-04   
           returns_industry_Non-Corporate                                 1.401431e-19   
           returns_industry_Non-Energy Materials                          2.065279e-04   
           returns_industry_Other                                         1.063496e-19   
           returns_industry_Technology                                    1.948145e-04   
           returns_industry_Telecommunications                            1.664834e-04   
           returns_industry_Utilities                                     9.790059e-05   
           returns_momentum                                               5.904250e-04   
           returns_size                                                   8.308090e-06   
           returns_value                                                  1.771054e-06   

                                                    returns_industry_Consumer Cyclicals  \
datetime   factor                                                                         
2024-01-31 returns_industry_Business Services                              2.585162e-04   
           returns_industry_Consumer Cyclicals                             6.957879e-04   
           returns_industry_Consumer Non-Cyclicals                        -8.832709e-03   
           returns_industry_Consumer Services                              6.648779e-04   
           returns_industry_Energy                                         1.203431e-03   
           returns_industry_Finance                                        4.447882e-04   
           returns_industry_Healthcare                                     2.886553e-04   
           returns_industry_Industrials                                    4.254043e-04   
           returns_industry_Non-Corporate                                  2.964223e-18   
           returns_industry_Non-Energy Materials                           1.812960e-04   
           returns_industry_Other                                          2.221192e-18   
           returns_industry_Technology                                     3.874619e-05   
           returns_industry_Telecommunications                             4.599082e-04   
           returns_industry_Utilities                                     -7.453745e-04   
           returns_momentum                                                1.343653e-02   
           returns_size                                                    4.664041e-04   
           returns_value                                                   2.914525e-04   

                                                    returns_industry_Consumer Non-Cyclicals  \
datetime   factor                                                                             
2024-01-31 returns_industry_Business Services                                 -2.642917e-04   
           returns_industry_Consumer Cyclicals                                -8.832709e-03   
           returns_industry_Consumer Non-Cyclicals                             2.487967e-01   
           returns_industry_Consumer Services                                 -9.042535e-03   
           returns_industry_Energy       

In [596]:
factorcovadjuster.calc_newey_west_frm( max_lags = 2, multiplier = 1)

returns_industry_Business Services  \
datetime   factor                                                                        
2024-01-31 returns_industry_Business Services                             3.636988e-05   
           returns_industry_Consumer Cyclicals                            4.011302e-05   
           returns_industry_Consumer Non-Cyclicals                       -4.255118e-05   
           returns_industry_Consumer Services                             4.588188e-05   
           returns_industry_Energy                                        4.904960e-05   
           returns_industry_Finance                                       4.219274e-05   
           returns_industry_Healthcare                                    1.812925e-05   
           returns_industry_Industrials                                   3.863008e-05   
           returns_industry_Non-Corporate                                 2.615170e-20   
           returns_industry_Non-Energy Materials                          3.268625e-05   
           returns_industry_Other                                         1.914887e-20   
           returns_industry_Technology                                    2.214432e-05   
           returns_industry_Telecommunications                            3.174126e-05   
           returns_industry_Utilities                                     2.235236e-05   
           returns_momentum                                               1.114545e-04   
           returns_size                                                  -7.094422e-06   
           returns_value                                                 -1.655034e-07   

                                                    returns_industry_Consumer Cyclicals  \
datetime   factor                                                                         
2024-01-31 returns_industry_Business Services                              4.011302e-05   
           returns_industry_Consumer Cyclicals                             4.860220e-05   
           returns_industry_Consumer Non-Cyclicals                        -5.115743e-05   
           returns_industry_Consumer Services                              5.565745e-05   
           returns_industry_Energy                                         3.697890e-05   
           returns_industry_Finance                                        4.258653e-05   
           returns_industry_Healthcare                                     1.668090e-05   
           returns_industry_Industrials                                    4.047293e-05   
           returns_industry_Non-Corporate                                  3.352919e-20   
           returns_industry_Non-Energy Materials                           3.304021e-05   
           returns_industry_Other                                          2.900477e-20   
           returns_industry_Technology                                     2.747970e-05   
           returns_industry_Telecommunications                             3.166747e-05   
           returns_industry_Utilities                                      2.648308e-05   
           returns_momentum                                                1.099419e-04   
           returns_size                                                   -1.030276e-05   
           returns_value                                                   5.144448e-07   

                                                    returns_industry_Consumer Non-Cyclicals  \
datetime   factor                                                                             
2024-01-31 returns_industry_Business Services                                 -4.255118e-05   
           returns_industry_Consumer Cyclicals                                -5.115743e-05   
           returns_industry_Consumer Non-Cyclicals                            -6.177953e-04   
           returns_industry_Consumer Services                                 -3.111829e-05   
           returns_industry_Energy       

In [597]:
type(factorcovadjuster.FRM)

pandas.core.frame.DataFrame

In [599]:
#.equals(test.cov().dropna())

In [600]:
test.cov().dropna()

,returns_industry_Business Services,returns_industry_Consumer Cyclicals,returns_industry_Consumer Non-Cyclicals,returns_industry_Consumer Services,returns_industry_Energy,returns_industry_Finance,returns_industry_Healthcare,returns_industry_Industrials,returns_industry_Non-Corporate,returns_industry_Non-Energy Materials,returns_industry_Other,returns_industry_Technology,returns_industry_Telecommunications,returns_industry_Utilities,returns_momentum,returns_size,returns_value
returns_industry_Business Services,2.076908e-04,2.585162e-04,-2.642917e-04,2.562457e-04,2.706373e-04,2.182466e-04,1.844511e-04,2.316459e-04,1.401431e-19,2.065279e-04,1.063496e-19,1.948145e-04,1.664834e-04,9.790059e-05,5.904250e-04,8.308090e-06,1.771054e-06
returns_industry_Consumer Cyclicals,2.585162e-04,6.957879e-04,-8.832709e-03,6.648779e-04,1.203431e-03,4.447882e-04,2.886553e-04,4.254043e-04,2.964223e-18,1.812960e-04,2.221192e-18,3.874619e-05,4.599082e-04,-7.453745e-04,1.343653e-02,4.664041e-04,2.914525e-04
returns_industry_Consumer Non-Cyclicals,-2.642917e-04,-8.832709e-03,2.487967e-01,-9.042535e-03,-2.495463e-02,-4.789607e-03,-1.079296e-03,-3.268914e-03,-8.137948e-17,2.901155e-03,-6.103660e-17,6.825772e-03,-7.137549e-03,2.515573e-02,-3.716309e-01,-1.334037e-02,-8.568731e-03
returns_industry_Consumer Services,2.562457e-04,6.648779e-04,-9.042535e-03,7.137778e-04,1.238296e-03,4.485555e-04,2.807854e-04,4.192537e-04,3.034635e-18,1.651107e-04,2.270437e-18,2.017959e-05,4.644822e-04,-7.728134e-04,1.372188e-02,4.771054e-04,2.983321e-04
returns_industry_Energy,2.706373e-04,1.203431e-03,-2.495463e-02,1.238296e-03,3.138139e-03,7.745602e-04,3.381396e-04,6.455882e-04,8.240796e-18,3.643558e-05,6.184981e-18,-4.500137e-04,9.287246e-04,-2.366189e-03,3.754261e-02,1.320383e-03,8.477575e-04
returns_industry_Finance,2.182466e-04,4.447882e-04,-4.789607e-03,4.485555e-04,7.745602e-04,3.532505e-04,2.120380e-04,3.162532e-04,1.625127e-18,1.830299e-04,1.223154e-18,7.272412e-05,3.160438e-04,-3.363825e-04,7.368434e-03,2.483678e-04,1.574120e-04
returns_industry_Healthcare,1.844511e-04,2.886553e-04,-1.079296e-03,2.807854e-04,3.381396e-04,2.120380e-04,2.948365e-04,2.388102e-04,4.125924e-19,1.948134e-04,3.068079e-19,2.203350e-04,1.958148e-04,1.503678e-05,1.775873e-03,5.121342e-05,2.274046e-05
returns_industry_Industrials,2.316459e-04,4.254043e-04,-3.268914e-03,4.192537e-04,6.455882e-04,3.162532e-04,2.388102e-04,3.348825e-04,1.135148e-18,2.278359e-04,8.535811e-19,1.561428e-04,2.826533e-04,-1.868562e-04,5.114820e-03,1.671632e-04,1.019982e-04
returns_industry_Non-Corporate,1.401431e-19,2.964223e-18,-8.137948e-17,3.034635e-18,8.240796e-18,1.625127e-18,4.125924e-19,1.135148e-18,2.669640e-32,-8.852342e-19,1.997518e-32,-2.173954e-18,2.390091e-18,-8.191932e-18,1.216096e-16,4.361107e-18,2.800300e-18
returns_industry_Non-Energy Materials,2.065279e-04,1.812960e-04,2.901155e-03,1.651107e-04,3.643558e-05,1.830299e-04,1.948134e-04,2.278359e-04,-8.852342e-19,3.199502e-04,-6.606527e-19,3.029681e-04,9.877003e-05,4.354817e-04,-4.100243e-03,-1.630280e-04,-1.094085e-04


In [442]:
window = 252
second_level_index = test.columns
first_level_index = test.index[window-1:]


In [447]:
cov_matries = []
for i in range(len(test)-window+1):
    cov_matries.append(cov_ewa(test.iloc[i:i+window,:].T.to_numpy()))
    #print(cov_ewa(test.iloc[i:i+window,:].T.to_numpy()))

In [456]:
arr = np.array(cov_matries)

In [449]:
len(cov_matries)

815

In [452]:
#index = pd.MultiIndex.from_tuples((first_level_index, second_level_index), names=["datetime", "factors"])

In [ ]:
# Reshape the 3D array into a 2D array with MultiIndex
reshaped_array = data_array_3d.reshape((-1, data_array_3d.shape[-1]))

# Create a MultiIndex for rows
multi_index = pd.MultiIndex.from_product([row_labels_level_1, row_labels_level_2], names=['Level_1', 'Level_2'])

# Convert the reshaped array to a DataFrame with MultiIndex
df_multiindex = pd.DataFrame(reshaped_array, index=multi_index, columns=column_labels)


In [454]:
multi_index = pd.MultiIndex.from_product([first_level_index, second_level_index], names=['datetime', 'factor'])


In [459]:
pd.DataFrame(arr.reshape((-1, arr.shape[-1])), index = multi_index, columns = second_level_index)

returns_industry_Business Services  \
datetime   factor                                                                        
2020-12-17 returns_industry_Business Services                                 0.000410   
           returns_industry_Consumer Cyclicals                                0.000450   
           returns_industry_Consumer Non-Cyclicals                            0.000329   
           returns_industry_Consumer Services                                 0.000483   
           returns_industry_Energy                                            0.000516   
...                                                                                ...   
2024-01-31 returns_industry_Telecommunications                                0.000085   
           returns_industry_Utilities                                         0.000074   
           returns_momentum                                                  -0.000016   
           returns_size                                                      -0.000014   
           returns_value                                                     -0.000007   

                                                    returns_industry_Consumer Cyclicals  \
datetime   factor                                                                         
2020-12-17 returns_industry_Business Services                                  0.000450   
           returns_industry_Consumer Cyclicals                                 0.000621   
           returns_industry_Consumer Non-Cyclicals                             0.000413   
           returns_industry_Consumer Services                                  0.000618   
           returns_industry_Energy                                             0.000600   
...                                                                                 ...   
2024-01-31 returns_industry_Telecommunications                                 0.000114   
           returns_industry_Utilities                                          0.000096   
           returns_momentum                                                   -0.000015   
           returns_size                                                       -0.000022   
           returns_value                                                      -0.000010   

                                                    returns_industry_Consumer Non-Cyclicals  \
datetime   factor                                                                             
2020-12-17 returns_industry_Business Services                                      0.000329   
           returns_industry_Consumer Cyclicals                                     0.000413   
           returns_industry_Consumer Non-Cyclicals                                 0.000321   
           returns_industry_Consumer Services                                      0.000432   
           returns_industry_Energy                                                 0.000455   
...                                                                                     ...   
2024-01-31 returns_industry_Telecommunications                                     0.000084   
           returns_industry_Utilities                                              0.000080   
           returns_momentum                                                       -0.000022   
           returns_size                                                           -0.000017   
           returns_value                                                          -0.000007   

                                                    returns_industry_Consumer Services  \
datetime   factor                                                                        
2020-12-17 returns_industry_Business Services                                 0.000483   
           returns_industry_Consumer Cyclicals                                0.000618   
           returns_industry_Consumer Non-Cyclicals                            0.000432   
           

In [557]:
#factorcovadjuster.FRM.rolling(window=window).cov()
#cov_ewa(test.T.to_numpy())

In [558]:
#factorcovadjuster.calc_fcm_raw(90)

In [310]:
# access data by one of the index
#print(df_factor_covar.xs('2020-12-17', level='datetime'))

#### 4.5 Calculate Idiosyncratic Returns variance matrix

In [601]:
df_loadings

,datetime,id,return_1d,value,size,industry_Business Services,industry_Consumer Cyclicals,industry_Consumer Non-Cyclicals,industry_Consumer Services,industry_Energy,...,industry_Industrials,industry_Non-Corporate,industry_Non-Energy Materials,industry_Other,industry_Technology,industry_Telecommunications,industry_Utilities,momentum,country,period
0,2020-01-01,B00FG1-R,0.000000,0.014547,-0.371694,0,0,0,0,1,...,0,0,0,0,0,0,0,-0.019683,1,2020
1,2020-01-01,B01HWF-R,0.000000,-0.442699,-1.358461,0,0,1,0,0,...,0,0,0,0,0,0,0,-0.151665,1,2020
2,2020-01-01,B04XY5-R,0.000000,-1.719625,-0.694700,0,0,0,0,0,...,0,0,1,0,0,0,0,0.031746,1,2020
3,2020-01-01,B06HD3-R,0.000000,-0.097482,-0.852544,0,0,0,0,0,...,0,0,0,0,0,0,0,0.050132,1,2020
4,2020-01-01,B0TXKG-R,0.000000,1.091488,0.566158,0,0,0,1,0,...,0,0,0,0,0,0,0,0.066271,1,2020
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3383982,2024-01-31,XQF8XY-R,0.004628,0.042221,1.488157,0,0,0,0,0,...,0,0,0,0,0,0,0,-0.022630,1,2024
3383983,2024-01-31,XQFML8-R,0.032251,-0.256071,0.135506,0,0,0,0,1,...,0,0,0,0,0,0,0,-0.040797,1,2024
3383984,2024-01-31,XQN3ZC-R,0.011494,-0.085134,-0.600385,0,0,0,0,0,...,0,0,0,0,0,0,0,0.025865,1,2024
3383985,2024-01-31,XR00QK-R,0.005945,0.613000,-0.403807,0,0,0,1,0,...,0,0,0,0,0,0,0,0.033170,1,2024


In [615]:
factor_cols = [
    col
    for col in df_loadings.columns if f"returns_{col}" in df_factor_returns.columns
    #if col in df_loadings.columns and col != "datetime"
]

# Merge the DataFrames on 'date'
df_idio = pd.merge(
    df_loadings, df_factor_returns, on="datetime", suffixes=("", "_factor_returns")
)

In [616]:
df_loadings.columns

Index(['datetime', 'id', 'return_1d', 'value', 'size',
       'industry_Business Services', 'industry_Consumer Cyclicals',
       'industry_Consumer Non-Cyclicals', 'industry_Consumer Services',
       'industry_Energy', 'industry_Finance', 'industry_Healthcare',
       'industry_Industrials', 'industry_Non-Corporate',
       'industry_Non-Energy Materials', 'industry_Other',
       'industry_Technology', 'industry_Telecommunications',
       'industry_Utilities', 'momentum', 'country', 'period'],
      dtype='object')

In [617]:
df_factor_returns.columns

Index(['datetime', 'r2', 't_values_industry_Business Services',
       't_values_industry_Consumer Cyclicals',
       't_values_industry_Consumer Non-Cyclicals',
       't_values_industry_Consumer Services', 't_values_industry_Energy',
       't_values_industry_Finance', 't_values_industry_Healthcare',
       't_values_industry_Industrials', 't_values_industry_Non-Corporate',
       't_values_industry_Non-Energy Materials', 't_values_industry_Other',
       't_values_industry_Technology', 't_values_industry_Telecommunications',
       't_values_industry_Utilities', 't_values_momentum', 't_values_size',
       't_values_value', 'returns_industry_Business Services',
       'returns_industry_Consumer Cyclicals',
       'returns_industry_Consumer Non-Cyclicals',
       'returns_industry_Consumer Services', 'returns_industry_Energy',
       'returns_industry_Finance', 'returns_industry_Healthcare',
       'returns_industry_Industrials', 'returns_industry_Non-Corporate',
       'returns_indu

In [618]:
df_idio.columns

Index(['datetime', 'id', 'return_1d', 'value', 'size',
       'industry_Business Services', 'industry_Consumer Cyclicals',
       'industry_Consumer Non-Cyclicals', 'industry_Consumer Services',
       'industry_Energy', 'industry_Finance', 'industry_Healthcare',
       'industry_Industrials', 'industry_Non-Corporate',
       'industry_Non-Energy Materials', 'industry_Other',
       'industry_Technology', 'industry_Telecommunications',
       'industry_Utilities', 'momentum', 'country', 'period', 'r2',
       't_values_industry_Business Services',
       't_values_industry_Consumer Cyclicals',
       't_values_industry_Consumer Non-Cyclicals',
       't_values_industry_Consumer Services', 't_values_industry_Energy',
       't_values_industry_Finance', 't_values_industry_Healthcare',
       't_values_industry_Industrials', 't_values_industry_Non-Corporate',
       't_values_industry_Non-Energy Materials', 't_values_industry_Other',
       't_values_industry_Technology', 't_values_indust

In [619]:
factor_cols

['value',
 'size',
 'industry_Business Services',
 'industry_Consumer Cyclicals',
 'industry_Consumer Non-Cyclicals',
 'industry_Consumer Services',
 'industry_Energy',
 'industry_Finance',
 'industry_Healthcare',
 'industry_Industrials',
 'industry_Non-Corporate',
 'industry_Non-Energy Materials',
 'industry_Other',
 'industry_Technology',
 'industry_Telecommunications',
 'industry_Utilities',
 'momentum']

In [623]:
# Multiplying matching columns
for col in factor_cols:
    df_idio[f"{col}_calc_f_r"] = df_idio[col] * df_idio[f"{col}_factor_returns"]

KeyError: 'value_factor_returns'

In [624]:
# # Dropping the extra columns
df_idio = df_idio.drop(columns=factor_cols)   # drop factor columns
df_idio = df_idio.drop(columns=[f"{col}_factor_returns" for col in factor_cols])   


KeyError: "['value_factor_returns', 'size_factor_returns', 'industry_Business Services_factor_returns', 'industry_Consumer Cyclicals_factor_returns', 'industry_Consumer Non-Cyclicals_factor_returns', 'industry_Consumer Services_factor_returns', 'industry_Energy_factor_returns', 'industry_Finance_factor_returns', 'industry_Healthcare_factor_returns', 'industry_Industrials_factor_returns', 'industry_Non-Corporate_factor_returns', 'industry_Non-Energy Materials_factor_returns', 'industry_Other_factor_returns', 'industry_Technology_factor_returns', 'industry_Telecommunications_factor_returns', 'industry_Utilities_factor_returns', 'momentum_factor_returns'] not found in axis"

In [625]:
df_idio.columns

Index(['datetime', 'id', 'return_1d', 'country', 'period', 'r2',
       't_values_industry_Business Services',
       't_values_industry_Consumer Cyclicals',
       't_values_industry_Consumer Non-Cyclicals',
       't_values_industry_Consumer Services', 't_values_industry_Energy',
       't_values_industry_Finance', 't_values_industry_Healthcare',
       't_values_industry_Industrials', 't_values_industry_Non-Corporate',
       't_values_industry_Non-Energy Materials', 't_values_industry_Other',
       't_values_industry_Technology', 't_values_industry_Telecommunications',
       't_values_industry_Utilities', 't_values_momentum', 't_values_size',
       't_values_value', 'returns_industry_Business Services',
       'returns_industry_Consumer Cyclicals',
       'returns_industry_Consumer Non-Cyclicals',
       'returns_industry_Consumer Services', 'returns_industry_Energy',
       'returns_industry_Finance', 'returns_industry_Healthcare',
       'returns_industry_Industrials', 'return

In [626]:
#### 4.3 Calc idiosyncratic risk

# factor_cols = [
#     col
#     for col in df_factor_returns.columns
#     if col in df_loadings.columns and col != "datetime"
# ]

# # Merge the DataFrames on 'date'
# df_idio = pd.merge(
#     df_loadings, df_factor_returns, on="datetime", suffixes=("", "_factor_returns")
# )

# # Multiplying matching columns
# for col in factor_cols:
#     df_idio[f"{col}_calc_f_r"] = df_idio[col] * df_idio[f"{col}_factor_returns"]

# # # Dropping the extra columns
# df_idio = df_idio.drop(columns=factor_cols)   # drop factor columns
# df_idio = df_idio.drop(columns=[f"{col}_factor_returns" for col in factor_cols])   

# Summing the specified columns
df_idio["factor_return_1d"] = df_idio[[f"{col}_calc_f_r" for col in factor_cols]].sum(
    axis=1
)
df_idio["factor_return_1d_error"] = df_idio["factor_return_1d"] - df_idio["return_1d"]

df_idio = df_idio[
    ["datetime", "id", "return_1d", "factor_return_1d", "factor_return_1d_error"]
]

In [627]:
df_idio

,datetime,id,return_1d,factor_return_1d,factor_return_1d_error
0,2020-01-01,B00FG1-R,0.000000,0.000000,0.000000
1,2020-01-01,B01HWF-R,0.000000,0.000000,0.000000
2,2020-01-01,B04XY5-R,0.000000,0.000000,0.000000
3,2020-01-01,B06HD3-R,0.000000,0.000000,0.000000
4,2020-01-01,B0TXKG-R,0.000000,0.000000,0.000000
...,...,...,...,...,...
3383982,2024-01-31,XQF8XY-R,0.004628,0.001224,-0.003404
3383983,2024-01-31,XQFML8-R,0.032251,0.016376,-0.015875
3383984,2024-01-31,XQN3ZC-R,0.011494,-0.013042,-0.024537
3383985,2024-01-31,XR00QK-R,0.005945,-0.002576,-0.008522


In [ ]:
# Calculate idiosync_return_var from return error
df_idio["idio_risk_252d"] = (
    df_idio.sort_values(by=["id", "datetime"])
    .groupby("id")["factor_return_1d_error"]
    .rolling(window=252)
    .var()
    .reset_index(level=0, drop=True)
)
df_idio = df_idio.set_index("datetime").pivot(columns="id", values="idio_risk_252d")


In [629]:
# Calculate idiosync_return_var from return error
df_idio.loc[:,"idio_risk_252d"] = (
    df_idio.sort_values(by=["id", "datetime"])
    .groupby("id")["factor_return_1d_error"]
    .rolling(window=252)
    .var()
    .reset_index(level=0, drop=True)
)
df_idio = df_idio.set_index("datetime").pivot(columns="id", values="idio_risk_252d")


KeyError: 'id'

In [630]:
df_idio

id,B00FG1-R,B01HWF-R,B04XY5-R,B06HD3-R,B0TXKG-R,B0VZ2Z-R,B0ZCK4-R,B12BZP-R,B14762-R,B15GK7-R,...,XQC8FD-R,XQCN4R-R,XQCWLZ-R,XQCXP0-R,XQF8XY-R,XQFML8-R,XQGHLF-R,XQN3ZC-R,XR00QK-R,XR7GZL-R
datetime,,,,,,,,,,,,,,,,,,,,,
2020-01-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-01-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-01-03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-01-06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-01-07,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-01-25,NaN,0.001568,NaN,0.000935,0.000272,0.000554,0.000272,NaN,0.000287,0.000529,...,0.000295,0.000125,0.013638,0.000293,0.000086,0.000512,NaN,0.000304,0.000287,0.000344
2024-01-26,NaN,0.001574,NaN,0.000937,0.000273,0.000553,0.000265,NaN,0.000287,0.000535,...,0.000295,0.000125,0.013639,0.000293,0.000086,0.000522,NaN,0.000307,0.000287,0.000343
2024-01-29,NaN,0.001575,NaN,0.000941,0.000273,0.000556,0.000267,NaN,0.000288,0.000536,...,0.000295,0.000125,0.013638,0.000293,0.000086,0.000491,NaN,0.000309,0.000287,0.000344
